In [ ]:
# define matrix A & b code
# import lib.
import numpy as np 

def mat_fun(n):
    
    # grid size
    L = 1
    h = L/(n-1)
    
    # define mat.
    A = np.zeros((n**2,n**2),dtype='float') 
    b = np.zeros(n**2,dtype='float')

    # top boundary
    for i in range(1,n):
        k = (n-1)*n + i
        A[k,k] = 1
        b[k] = 1  
        
    # left boundary
    for j in range(n):
        k = j*n + 0
        A[k,k] = 1
        b[k] = 0
        
    # bottom boundary
    for i in range(1,n):
        k = 0*n + i
        A[k,k] = 1
        b[k] = 1
        
    # right boundary
    for j in range(1,n-1):
        k = j*n + n-1
        A[k,k] = -1
        A[k,k-1] = 1
        b[k] = 0
        
    # interior points
    for j in range(1,n-1):
        for i in range(1,n-1):
            k = j*n + i 
            A[k,k] = -4/h**2 
            A[k,k-1] = 1/h**2
            A[k,k+1] = 1/h**2
            A[k,k+n] = 1/h**2
            A[k,k-n] = 1/h**2
            b[k] = 0 
        
    return A,b

In [ ]:
# normal gauss seidel
# import lib.
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import Ab_fun

# grid size
n = 10
L = 1
h = L/(n-1)

# define mat.
A, b = Ab_fun.Ab_fun(n)
x = np.zeros((n,n),dtype='float')
x_cor = np.linspace(0,L,n)
y_cor = np.linspace(0,L,n)
x_mgrid, y_mgrid = np.meshgrid(x_cor,y_cor)

# left boundary
for j in range(n):
    k = j*n + 0
    x[0,j] = b[k]
    
# top boundary
for i in range(1,n):
    k = (n-1)*n + i
    x[i,n-1] = b[k]
    
# bottom boundary
for i in range(1,n):
    k = 0*n + i
    x[i,0] = b[k]

# sparse gaussSeidel
A_sp = sparse.coo_matrix(A, shape=(n**2, n**2)) # AA, JR, JC
AA = A_sp.data
JR = A_sp.row
JC = A_sp.col

res_vct = np.zeros(n**2)
res_nrm = 1
tol = 1e-5
itr_cnt = 1
omega = 1.25
res_nrm_vct = []
while(res_nrm>tol):
    
    # right boundary
    i = n-1
    for j in range(1,n-1):
        k = j*n + i
        x[i,j] = omega*(b[k]-A[k,k-1]*x[i-1,j]-A[k,k-n]*x[i,j-1]-\
                        A[k,k+n]*x[i,j+1])/A[k,k]+(1-omega)*x[i,j]
    # interior
    for j in range(1,n-1):
        for i in range(1,n-1):
            k = j*n + i 
            x[i,j] = omega*(b[k]-A[k,k+1]*x[i+1,j]-A[k,k-1]*x[i-1,j]-\
                            A[k,k+n]*x[i,j+1]-A[k,k-n]*x[i,j-1])/A[k,k]+\
                            (1-omega)*x[i,j]    
    
    # residual
    for j in range(1,n-1):
        for i in range(1,n-1):
            k = j*n + i
            res_vct[k] = b[k]-(A[k,k]*x[i,j]+A[k,k-1]*x[i-1,j]+\
                         A[k,k+1]*x[i+1,j]+A[k,k-n]*x[i,j-1]+A[k,k+n]*x[i,j+1])
    i = n-1
    for j in range(1,n-1):
        k = j*n + i
        res_vct[k] = b[k]-(A[k,k]*x[i,j]+A[k,k-1]*x[i-1,j]+A[k,k-n]*x[i,j-1]+\
                     A[k,k+n]*x[i,j+1])
    # convergence
    itr_cnt = itr_cnt + 1
    res_nrm = np.linalg.norm(res_vct)
    print(res_nrm)
    res_nrm_vct.append(res_nrm)
    
x6 = x
res_nrm_vct6 = res_nrm_vct
itr_cnt6 = itr_cnt
itr6 = np.arange(1,itr_cnt6,1)

# plot result
plt.loglog(itr6,res_nrm_vct6,itr5,res_nrm_vct5,itr4,res_nrm_vct4,itr3,res_nrm_vct3)
plt.title("The effect of the omega on the convergence of the Gauss Seidel algorithm")
plt.legend(('w=1.25','w=1','w=0.75','w=0.5'))
plt.ylim([10**-6,10**4])
plt.grid()
plt.xlabel('Iteration')
plt.ylabel('Residual')


In [ ]:
# gauss siedel for coordinate format
# import lib.
import numpy as np
import Get_Element as GE

def sparse_gs (A_sp,b,x,n,omega):
    res_vct = np.zeros(n**2) 
    res_nrm = 1 
    tol = 1e-5 # tolerance
    itr_cnt = 1 
    res_nrm_vct = [] 
    while(res_nrm>tol):
        # Gauss-Seidel equation
        for j in range(1,n-1):
            for i in range(1,n-1):
                k = j*n + i
                E = GE.Get_Element(A_sp,k,k+1)[0]
                W = GE.Get_Element(A_sp,k,k-1)[0]
                N = GE.Get_Element(A_sp,k,k+n)[0]
                S = GE.Get_Element(A_sp,k,k-n)[0]
                C = GE.Get_Element(A_sp,k,k)[0] 
                x[i,j] = omega*(b[k]-E*x[i+1,j]-W*x[i-1,j]-N*x[i,j+1]-S*x[i,j-1])/C+(1-omega)*x[i,j]
        # right boundary
        i = n-1
        for j in range(1,n-1):
            k = j*n + i
            W = GE.Get_Element(A_sp,k,k-1)[0]
            N = GE.Get_Element(A_sp,k,k+n)[0]
            S = GE.Get_Element(A_sp,k,k-n)[0]
            C = GE.Get_Element(A_sp,k,k)[0]
            x[i,j] = omega*(b[k]-W*x[i-1,j]-S*x[i,j-1]- N*x[i,j+1])/C+(1-omega)*x[i,j]
            
        # residual
        for j in range(1,n-1):
            for i in range(1,n-1):
                k = j*n + i
                E = GE.Get_Element(A_sp,k,k+1)[0]
                W = GE.Get_Element(A_sp,k,k-1)[0]
                N = GE.Get_Element(A_sp,k,k+n)[0]
                S = GE.Get_Element(A_sp,k,k-n)[0]
                C = GE.Get_Element(A_sp,k,k)[0]
                res_vct[k] = b[k]-(C*x[i,j]+W*x[i-1,j]+E*x[i+1,j]+S*x[i,j-1]+N*x[i,j+1])
        i = n-1
        for j in range(1,n-1):
            k = j*n + i
            E = GE.Get_Element(A_sp,k,k+1)[0]
            W = GE.Get_Element(A_sp,k,k-1)[0]
            N = GE.Get_Element(A_sp,k,k+n)[0]
            S = GE.Get_Element(A_sp,k,k-n)[0]
            C = GE.Get_Element(A_sp,k,k)[0]
            res_vct[k] = b[k]-(C*x[i,j]+W*x[i-1,j]+S*x[i,j-1]+N*x[i,j+1])
            
        # convergence
        itr_cnt = itr_cnt + 1
        res_nrm = np.linalg.norm(res_vct)
        print(res_nrm)
        res_nrm_vct.append(res_nrm)
    return x,res_nrm_vct

In [ ]:
# main gauss siedel
# import lib.
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import Ab_module
import sparse_gs as sp_gs

# grid size
n = 10 
L = 1
h = L/(n-1)

# define mat.
A, b = Ab_module.Ab_fun(n)
x = np.zeros((n,n),dtype='float')
x_cor = np.linspace(0,L,n)
y_cor = np.linspace(0,L,n)
x_mgrid, y_mgrid = np.meshgrid(x_cor,y_cor)

# top boundary
for i in range(1,n):
    k = (n-1)*n + i
    x[i,n-1] = b[k]
    
# left boundary
for j in range(n):
    k = j*n + 0
    x[0,j] = b[k]    

# bottom boundary
for i in range(1,n):
    k = 0*n + i
    x[i,0] = b[k]

# sparsity check
plt.spy(A,markersize=3,markerfacecolor='blue')
plt.xlim([0,100])
plt.ylim([100,0])
plt.xlabel('j',fontsize=16)
plt.ylabel('i',fontsize=16)
plt.grid()
plt.title("sparsity check at nx=ny=10")
plt.show()

# solution
A_sp = sparse.coo_matrix(A, shape=(n**2, n**2)) # AA, JR, JC
final_sol, final_res_vct = sp_gs.sparse_gs(A_sp,b,x,n,1)

# post processing
itr_cnt = len(final_res_vct)
cf = plt.contourf(x_mgrid,y_mgrid,np.matrix.transpose(final_sol))
plt.set_cmap('jet')
plt.colorbar(cf)
plt.xlabel('x')
plt.ylabel('y')
plt.title("nx=ny=10")
plt.show()
plt.loglog(np.arange(1,itr_cnt+1,1),final_res_vct)
plt.show()

In [ ]:
# grmres without preconditioning
# import lib.
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse.linalg as spla
import Ab_module
import make_callback

# grid size
n = 10 
L = 1 
h = L/(n-1) 

# define mat.
A, b = Ab_module.Ab_fun(n) 
x = np.zeros(n**2,dtype='float') 
x_cor = np.linspace(0,L,n)
y_cor = np.linspace(0,L,n)
x_mgrid4, y_mgrid4 = np.meshgrid(x_cor,y_cor)

# left boundary
for j in range(n):
    k = j*n + 0
    x[k] = b[k]
# top boundary
for i in range(1,n):
    k = (n-1)*n + i
    x[k] = b[k]
# bottom boundary
for i in range(1,n):
    k = 0*n + i
    x[k] = b[k]
    
# solve Ax=b
sol_gm = spla.gmres(A,b,x0=x,tol=1e-05,atol=1e-5,
                    callback = make_callback.make_callback(),
                    maxiter=25000);
x_arr4 = np.asarray(sol_gm[0])
sol4 = np.reshape(x_arr4,(n,n))

# post processing
cf = plt.contourf(x_mgrid3,y_mgrid3,sol3)
plt.set_cmap('jet')
plt.colorbar(cf)
plt.xlabel('x')
plt.ylabel('y')
plt.title("nx=ny=10")

res_arr4 = np.array(make_callback.make_callback.res)
cnt = len(res_arr4)
plt.loglog(np.arange(1,cnt+1,1),res_arr4)

# convergence
itr4 = np.arange(1,len(res_arr4)+1,1)
itr3 = np.arange(1,len(res_arr3)+1,1)
itr2 = np.arange(1,len(res_arr2)+1,1)
itr1 = np.arange(1,len(res_arr1)+1,1)
plt.loglog(itr4,res_arr4,itr3,res_arr3,itr2,res_arr2,itr1,res_arr1)
plt.xlabel('Iteration')
plt.ylabel('Residual')
plt.grid()
plt.legend(('nx=ny=10','nx=ny=20','nx=ny=50','nx=ny=100'))

In [ ]:
# grmres with preconditioning
# import lib.
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse.linalg as spla
import scipy.sparse as sparse
import Ab_module
import make_callback

# grid size
n = 10 
L = 1 
h = L/(n-1) 

# define mat.
A, b = Ab_module.Ab_fun(n) 
x = np.zeros(n**2,dtype='float') 
x_cor = np.linspace(0,L,n)
y_cor = np.linspace(0,L,n)
x_mgrid4, y_mgrid4 = np.meshgrid(x_cor,y_cor)

# bottom boundary
for i in range(1,n):
    k = 0*n + i
    x[k] = b[k]

# left boundary
for j in range(n):
    k = j*n + 0
    x[k] = b[k]

# top boundary
for i in range(1,n):
    k = (n-1)*n + i
    x[k] = b[k]

# preconditioning
A2 = sparse.csc_matrix(A)
M2 = spla.spilu(A2,fill_factor=20.0)
M_x = lambda x: M2.solve(x)
M = spla.LinearOperator((n**2,n**2),M_x)

# solve Ax=b
sol_gm = spla.gmres(A,b,x0=x,tol=1e-05,M=M,atol=1e-5,callback=make_callback.make_callback());
x_arr4 = np.asarray(sol_gm[0])
sol4 = np.reshape(x_arr4,(n,n))
res_arr4 = np.array(make_callback.make_callback.res)

# plot convergence
itr4 = np.arange(1,len(res_arr4)+1,1)
itr3 = np.arange(1,len(res_arr3)+1,1)
itr2 = np.arange(1,len(res_arr2)+1,1)
itr1 = np.arange(1,len(res_arr1)+1,1)
plt.loglog(itr4,res_arr4,itr3,res_arr3,itr2,res_arr2,itr1,res_arr1)
plt.xlim([1, 10**2])
plt.legend(('nx=ny=10','nx=ny=20','nx=ny=50','nx=ny=100'))
plt.xlabel('Iteration')
plt.ylabel('Residual')
plt.grid()

# post processing
cf = plt.contourf(x_mgrid1,y_mgrid1,sol1)
plt.set_cmap('jet')
plt.colorbar(cf)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(('nx=ny=10','nx=ny=20','nx=ny=50','nx=ny=100'))

In [ ]:
# callback

def make_callback():
    myres = []
    closure_variables = dict(counter=0, residuals=[])
    make_callback.res = myres
    
    def callback(residuals):
        closure_variables["counter"] += 1
        closure_variables["residuals"].append(residuals)
        myres.append(residuals)
        print (closure_variables["counter"], residuals)
    return callback